In [1]:
import os

# Move to Thesis directory (two levels up)
os.chdir(os.path.abspath(os.path.join("..", "..")))

# Move to model/src if it exists
model_dir = os.path.join(os.getcwd(), "model", "src")
if os.path.exists(model_dir):
    os.chdir(model_dir)

print("Current Directory:", os.getcwd())


Current Directory: c:\Users\1176153\Downloads\github\Thesis\model\src


# Imports

In [3]:
import selenium
import requests
import pandas as pd
import numpy as np 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import libs.data_extraction as de




## Loading dictionary with all the program links

In [4]:
dict_programs_links = pd.read_pickle(r"../../data/Webscrapping/programs_links.pkl")
dict_programs_links

{'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/': ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/'],
 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/': ['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-p

In [18]:
dict_programs_links_copy = dict_programs_links.copy()

In [9]:

# The new course links to be added
new_links = [
    "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-data-driven-marketing-with-a-specialization-in-data-science-for-marketing-working-hours-format/",
    "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-data-driven-marketing-with-a-specialization-in-digital-marketing-and-analytics-working-hours-format/",
    "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-data-driven-marketing-with-a-specialization-in-marketing-intelligence-working-hours-format/",
    "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-information-management-with-a-specialization-in-information-systems-management-working-hours-format/"
]

# Add these new links to the 'postgraduate-programs-and-master-degree-programs' key
dict_programs_links['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/'].extend(new_links)

# Print the updated list of links to verify
print(dict_programs_links['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/'])

['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence-and-analytics-for-hospitality-tourism/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-analytics/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-science-for-finance/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-science-for-marketing/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-digital-enterprise-management/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-p

In [10]:
values_to_remove = {'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-data-driven-marketing-with-a-specialization-in-data-science-for-marketing/', 
                    'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-data-driven-marketing-with-a-specialization-in-digital-marketing-and-analytics/',
                    'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-data-driven-marketing-with-a-specialization-in-marketing-intelligence/',
                    'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-in-information-management-with-a-specialization-in-information-systems-management/'}



In [11]:
# Remove the values from the lists inside the dictionary
dict_programs_links = {
    k: [link for link in v if link not in values_to_remove]  # Filter out unwanted values
    for k, v in dict_programs_links.items()
}

# Optionally, remove keys with empty lists after filtering
dict_programs_links = {k: v for k, v in dict_programs_links.items() if v}

In [12]:
if len(dict_programs_links) > 1:
    # Convert dictionary items to a list and get the second key-value pair
    items = list(dict_programs_links.items())
    second_url, second_filtered_links = items[1]
    
    # Print the second URL and filtered links
    print(f"Second URL: {second_url}")
    print(f"Filtered links for the second URL: {second_filtered_links}")
elif dict_programs_links:
    print("The dictionary does not have a second item.")
else:
    print("The dictionary is empty.")

Second URL: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/
Filtered links for the second URL: ['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence-and-analytics-for-hospitality-tourism/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-analytics/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-science-for-finance/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-science-for-marketing/', 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-an

## Exploration of the postgraduate and masters webpage design

### Postgraduate and Masters

**POSTGRADUATE OLD WAY:**

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

first_url = second_filtered_links[0]

# Open webpage
driver.get(first_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")


'''
# REMOVE THE BROCHURE AND COUNTRIES SECTIONS
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  

# REMOVING THE STUDY PLAN (1), TEACHING STAFF (2), AND ACADEMIC CALENDER (3) SECTIONS
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [0, 1, 2]  # 0-based index for 1th, 2th, and 3th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()
    
'''
# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)


**MASTERS OLD WAY:**

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

first_url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-data-science/"

# Open webpage
driver.get(first_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# REMOVE THE BROCHURE AND COUNTRIES SECTIONS
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  

# REMOVING THE STUDY PLAN (1), TEACHING STAFF (2), AND ACADEMIC CALENDER (3) SECTIONS
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [0, 1, 2]  # 0-based index for 1th, 2th, and 3th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)

**Able to extract information from content_article effectlively by using the Xpath**

In [13]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"


# Open webpage
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

# Find the main div using full XPath
try:
    main_block = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/main/div[5]")))
    articles = main_block.find_elements(By.XPATH, ".//section/div/article")

    if articles:
        for i, article in enumerate(articles, 1):
            try:
                # Locate the h2 element inside each article
                heading = article.find_element(By.XPATH, ".//div[2]/h2")
                print(f"Article {i}: {heading.text.strip()}")
            except:
                print(f"Article {i}: No heading found")
    else:
        print("No articles found inside 'block has-aside'.")
except:
    print("Main block with class 'block has-aside' not found.")

# Close WebDriver
driver.quit()


Article 1: Study plan
Article 2: Program Coordinator
Article 3: Faculty
Article 4: Academic calendar
Article 5: Length, timetable and exams
Article 6: Admissions and fees
Article 7: Scholarships
Article 8: Achievement awards
Article 9: FAQ


**Able to extract information from content_article effectlively by using the Class_name**

In [24]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

# Find the main div dynamically
try:
    main_block = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "block.has-aside")))
    articles = main_block.find_elements(By.CLASS_NAME, "content__article")

    if articles:
        for article in articles:
            try:
                # Find the article's <h2> heading
                heading_element = article.find_element(By.CLASS_NAME, "content__heading")
                heading_text = heading_element.text.strip()

                # Extract text only for a specific article name (modify this string to search for a different one)
                search_name = "Faculty"  # Change this to the article title you want
                if search_name in heading_text:
                    print(f"Found Article: {heading_text}")

                    # Extract full text inside this article
                    article_text = article.text.strip()
                    print("\n--- Article Content ---\n", article_text)
                    break  # Stop after finding the first match
            except:
                pass
    else:
        print("No articles found inside 'block has-aside'.")
except:
    print("Main block with class 'block has-aside' not found.")

# Close WebDriver
driver.quit()



Found Article: Faculty

--- Article Content ---
 Faculty
Afshin Ashofteh
Assistant Professor
aashofteh@novaims.unl.pt
Know more
Bernardo Dias Raimundo
Adjunct Lecturer
braimundo@novaims.unl.pt
Know more
Hugo Borginho
Adjunct Lecturer
hborginho@novaims.unl.pt
Know more
Jorge Bravo
Associate Professor
jbravo@novaims.unl.pt
Know more
Lourdes Afonso
Invited Associate Professor
mafonso@novaims.unl.pt
Know more
Luís Oliveira
Professor of the Practice
loliveira@novaims.unl.pt
Know more
See here the detailed list of the Program Faculty members


**Able to extract information from content_article effectlively by using the CSS_SELECTOR - BETTER!**

In [21]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

try:
    # Find the main div dynamically
    main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))
    
    # Find all articles within the main block
    articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

    search_name = "Faculty"  # Modify this for different sections

    if articles:
        for article in articles:
            try:
                # Find the <h2> heading inside the article
                heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                heading_text = heading_element.text.strip()

                if search_name in heading_text:
                    print(f"✅ Found Article: {heading_text}")

                    # Extract the corresponding content within <div class="content__article-wrap">
                    content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                    content_text = content_element.text.strip()

                    print("\n--- Extracted Content ---\n", content_text)
                    break  # Stop after finding the first match

            except Exception as e:
                print(f"Skipping article due to error: {e}")

    else:
        print("No articles found inside 'block has-aside'.")

except Exception as e:
    print(f"Error locating elements: {e}")

# Close WebDriver
driver.quit()




✅ Found Article: Faculty

--- Extracted Content ---
 Faculty
Afshin Ashofteh
Assistant Professor
aashofteh@novaims.unl.pt
Know more
Bernardo Dias Raimundo
Adjunct Lecturer
braimundo@novaims.unl.pt
Know more
Hugo Borginho
Adjunct Lecturer
hborginho@novaims.unl.pt
Know more
Jorge Bravo
Associate Professor
jbravo@novaims.unl.pt
Know more
Lourdes Afonso
Invited Associate Professor
mafonso@novaims.unl.pt
Know more
Luís Oliveira
Professor of the Practice
loliveira@novaims.unl.pt
Know more
See here the detailed list of the Program Faculty members


### Faculty and Study Plan - Extracting every information and links inside 

In [38]:


# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

try:
    # Find the main div dynamically
    main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))

    # Find all articles within the main block
    articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

    search_name = "Study plan"  # Modify this for different sections

    if articles:
        for article in articles:
            try:
                # Find the <h2> heading inside the article
                heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                heading_text = heading_element.text.strip()

                if search_name in heading_text:
                    print(f"✅ Found Article: {heading_text}")

                    # Extract the corresponding content within <div class="content__article-wrap">
                    content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                    content_html = content_element.get_attribute('outerHTML')

                    # Parse the content using BeautifulSoup
                    soup = BeautifulSoup(content_html, 'html.parser')

                    # Extract the text from the content (excluding links)
                    content_text = soup.get_text().strip()
                    print(f"--- Extracted Text ---\n{content_text}")

                    # Find all links inside the content and print them
                    links = soup.find_all('a', href=True)  # Find all links with an href attribute
                    if links:
                        print("\n--- Extracted Links ---")
                        for link in links:
                            link_url = link['href']
                            print(f"Found link: {link_url}")
                    else:
                        print("No links found in this article.")

                    break  # Stop after finding the first match

            except Exception as e:
                print(f"Skipping article due to error: {e}")

    else:
        print("No articles found inside 'block has-aside'.")

except Exception as e:
    print(f"Error locating elements: {e}")

# Close WebDriver
driver.quit()


✅ Found Article: Study plan
--- Extracted Text ---
Study plan
The study plan of this Program corresponds to 60 ECTS, of which 45 are in mandatory course units:

Mandatory course units

Banking and Insurance Economics
Banking and Insurance Regulation and Supervision
Credit Risk Management
Financial Derivatives and Risk Management
Investments and Portfolio Management
Life Insurance
Market and Liquidity Risk Management
Non-Life Insurance
Predictive Analytics in Finance





Course Unit







Loading...


modal item





card item








The remaining 15 ECTS correspond to the elective curricular units, to be chosen by each student from a wide range of course units available.



                    See here the detailed study plan

--- Extracted Links ---
Found link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/study-plan/


#### Extracting only the link to know the full study plan or teaching staff/faculty

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

try:
    # Find the main div dynamically
    main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))

    # Find all articles within the main block
    articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

    search_name = "Faculty"  # Modify this for different sections

    if articles:
        for article in articles:
            try:
                # Find the <h2> heading inside the article
                heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                heading_text = heading_element.text.strip()

                if search_name in heading_text:
                    print(f"✅ Found Article: {heading_text}")

                    # Extract the corresponding content within <div class="content__article-wrap">
                    content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                    content_html = content_element.get_attribute('outerHTML')

                    # Parse the content using BeautifulSoup
                    soup = BeautifulSoup(content_html, 'html.parser')

                    # Find all links inside the content and extract the last one
                    links = soup.find_all('a', href=True)  # Find all links with an href attribute
                    if links:
                        last_link = links[-1]  # Get the last link
                        last_link_url = last_link['href']
                        print(f"✅ Last Found Link: {last_link_url}")
                    else:
                        print("No links found in this article.")

                    break  # Stop after finding the first match

            except Exception as e:
                print(f"Skipping article due to error: {e}")

    else:
        print("No articles found inside 'block has-aside'.")

except Exception as e:
    print(f"Error locating elements: {e}")

# Close WebDriver
driver.quit()


✅ Found Article: Faculty
✅ Last Found Link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/faculty/


#### Checking if Faculty or Study plan exist if not extract the content from the original page

In [2]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/"
#url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

try:
    '''
    # Find the main div dynamically
    main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))
    print(f"✅ Found Main Block: {main_block}")
    # Find all articles within the main block'
    '''
    main_blocks = driver.find_elements(By.CSS_SELECTOR, "div.block.has-aside")
    if not main_blocks:
        print("No elements found for 'div.block.has-aside'. Exiting script.")
        driver.quit()
        exit()
    main_block = main_blocks[0]



    articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

    search_name = "Study plan"  # Modify this for different sections

    if articles:
        for article in articles:
            try:
                # Find the <h2> heading inside the article
                heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                heading_text = heading_element.text.strip()

                if search_name in heading_text:
                    print(f"✅ Found Article: {heading_text}")

                    # Extract the corresponding content within <div class="content__article-wrap">
                    content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                    content_html = content_element.get_attribute('outerHTML')

                    # Parse the content using BeautifulSoup
                    soup = BeautifulSoup(content_html, 'html.parser')

                    # Extract the text while making sure to handle the concatenation issue
                    content_text = soup.get_text(separator=' ', strip=True)
                    
                    # Ensure the spaces are added correctly and the text is properly divided
                    #print(f"--- Extracted Content ---\n{content_text}")

                    # Find all links inside the content and extract the last one
                    links = soup.find_all('a', href=True)  # Find all links with an href attribute
                    if links:
                        last_link = links[-1]  # Get the last link
                        last_link_url = last_link['href']
                        print(f"✅ Last Found Link: {last_link_url}")

                        # Open the link in the browser
                        driver.get(last_link_url)
                        time.sleep(5)  # Wait for the page to load

                    # If the search_name is "Faculty", check if the section <div class="block has-aside"> exists
                        if search_name == "Faculty":
                            try:
                                section = driver.find_element(By.CSS_SELECTOR, "div.block.has-aside")
                                print("✅ Found section <div class='block has-aside'> in the linked page.")

                                # Extract content using BeautifulSoup
                                page_html = driver.page_source
                                page_soup = BeautifulSoup(page_html, 'html.parser')

                                # Extract everything inside the <div class="block has-aside">
                                section_content = page_soup.find('div', class_='block has-aside')
                                print("\n--- Extracted Section Content ---")
                                print(section_content.get_text(separator=' ', strip=True))

                            except Exception as e:
                                print("Section <div class='block has-aside'> not found. Extracting content from the first page.")
                                # If the section is not found, extract the content from the first page
                                print(f"--- Extracted Content from the first page ---\n{content_text}")
                        else:
                            # If search_name is "Study plan", extract the entire content of the linked page
                            page_html = driver.page_source
                            page_soup = BeautifulSoup(page_html, 'html.parser')

                            # Extract the content from the linked page without checking for the section
                            print(f"--- Extracted Content from the linked page for {search_name} ---")
                            print(page_soup.get_text(separator=' ', strip=True))

                    else:
                        print("No links found in this article.")

                    break  # Stop after finding the first match for this search_name

            except Exception as e:
                print(f"Skipping article due to error: {e}")

    else:
        print("No articles found inside 'block has-aside'.")

except Exception as e:
    print(f"Error locating elements: {e}")

# Close WebDriver
driver.quit()


✅ Found Article: Study plan
✅ Last Found Link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/study-plan/
--- Extracted Content from the linked page for Study plan ---
Study Plan | NOVA IMS Your privacy choices Your personal data will be processed, and information from your device (cookies, unique identifiers and other device data) may be stored, accessed and shared with 848 TCF partners and other advertising partners or used specifically by this website or app. Some providers may process your personal data based on legitimate interest. You can object to this processing by managing the options below. Look for a link at the bottom of this page to manage or withdraw consent in your privacy and cookie settings. Purposes Store and/or access information on a device Develop and improve services Personalised advertising and content, advertising and content measurement, and audience research Specia

In [1]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/"
#url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

try:
    '''
    # Find the main div dynamically
    main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))
    print(f"✅ Found Main Block: {main_block}")
    # Find all articles within the main block'
    '''
    main_blocks = driver.find_elements(By.CSS_SELECTOR, "div.block.has-aside")
    if not main_blocks:
        print("No elements found for 'div.block.has-aside'. Exiting script.")
        driver.quit()
        exit()
    main_block = main_blocks[0]



    articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

    search_name = "Faculty"  # Modify this for different sections

    if articles:
        for article in articles:
            try:
                # Find the <h2> heading inside the article
                heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                heading_text = heading_element.text.strip()

                if search_name in heading_text:
                    print(f"✅ Found Article: {heading_text}")

                    # Extract the corresponding content within <div class="content__article-wrap">
                    content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                    content_html = content_element.get_attribute('outerHTML')

                    # Parse the content using BeautifulSoup
                    soup = BeautifulSoup(content_html, 'html.parser')

                    # Extract the text while making sure to handle the concatenation issue
                    content_text = soup.get_text(separator=' ', strip=True)
                    
                    # Ensure the spaces are added correctly and the text is properly divided
                    #print(f"--- Extracted Content ---\n{content_text}")

                    # Find all links inside the content and extract the last one
                    links = soup.find_all('a', href=True)  # Find all links with an href attribute
                    if links:
                        last_link = links[-1]  # Get the last link
                        last_link_url = last_link['href']
                        print(f"✅ Last Found Link: {last_link_url}")

                        # Open the link in the browser
                        driver.get(last_link_url)
                        time.sleep(5)  # Wait for the page to load

                    # If the search_name is "Faculty", check if the section <div class="block has-aside"> exists
                        if search_name == "Faculty":
                            try:
                                section = driver.find_element(By.CSS_SELECTOR, "div.block.has-aside")
                                print("✅ Found section <div class='block has-aside'> in the linked page.")

                                # Extract content using BeautifulSoup
                                page_html = driver.page_source
                                page_soup = BeautifulSoup(page_html, 'html.parser')

                                # Extract everything inside the <div class="block has-aside">
                                section_content = page_soup.find('div', class_='block has-aside')
                                print("\n--- Extracted Section Content ---")
                                print(section_content.get_text(separator=' ', strip=True))

                            except Exception as e:
                                print("Section <div class='block has-aside'> not found. Extracting content from the first page.")
                                # If the section is not found, extract the content from the first page
                                print(f"--- Extracted Content from the first page ---\n{content_text}")
                        else:
                            # If search_name is "Study plan", extract the entire content of the linked page
                            page_html = driver.page_source
                            page_soup = BeautifulSoup(page_html, 'html.parser')

                            # Extract the content from the linked page without checking for the section
                            print(f"--- Extracted Content from the linked page for {search_name} ---")
                            print(page_soup.get_text(separator=' ', strip=True))

                    else:
                        print("No links found in this article.")

                    break  # Stop after finding the first match for this search_name

            except Exception as e:
                print(f"Skipping article due to error: {e}")

    else:
        print("No articles found inside 'block has-aside'.")

except Exception as e:
    print(f"Error locating elements: {e}")

# Close WebDriver
driver.quit()

✅ Found Article: Faculty
✅ Last Found Link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/teaching-staff/
✅ Found section <div class='block has-aside'> in the linked page.

--- Extracted Section Content ---
Teaching Staff Teaching Staff Apply here Teaching Staff Bruno Jardim Assistant Professor bjardim@novaims.unl.pt Know more Bruno Rodrigues Adjunct Lecturer brodrigues@novaims.unl.pt Know more Dhruv Akshay Pandit Invited Teaching Assistant dpandit@novaims.unl.pt Know more Duarte Rodrigues Adjunct Lecturer duarte.rodrigues@novaims.unl.pt Know more Farina Pontejos Invited Teaching Assistant fpontejos@novaims.unl.pt Know more Fernando Bação Full Professor bacao@novaims.unl.pt Know more Frederico Cruz Jesus Associate Professor fjesus@novaims.unl.pt Know more Guilherme Victorino Assistant Professor gmvictorino@novaims.unl.pt Know more Ivo Bernardo Adjunct Lecturer ibernardo@novaims.unl.pt Know

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/"
driver.get(url)

# Wait for elements to load
wait = WebDriverWait(driver, 10)

#search_name = "Faculty"  # Modify this for different sections
search_name = "Study plan" 

# Directory to save extracted text files
postgraduates_masters_dir = r"..\..\data\Webscrapping\test"
os.makedirs(postgraduates_masters_dir, exist_ok=True)  # Ensure directory exists

try:
    # Find the main div dynamically
    main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))

    # Find all articles within the main block
    articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

    if articles:
        for article in articles:
            try:
                # Find the <h2> heading inside the article
                heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                heading_text = heading_element.text.strip()
                
                if search_name in heading_text:
                    print(f"✅ Found Article: {heading_text}")

                    # Extract the corresponding content within <div class="content__article-wrap">
                    content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                    content_html = content_element.get_attribute('outerHTML')

                    # Parse the content using BeautifulSoup
                    soup = BeautifulSoup(content_html, 'html.parser')

                    # Extract the text while making sure to handle the concatenation issue
                    content_text = soup.get_text(separator=' ', strip=True)
                    
                    # Ensure the spaces are added correctly and the text is properly divided
                    #print(f"--- Extracted Content ---\n{content_text}")

                    # Find all links inside the content and extract the last one
                    links = soup.find_all('a', href=True)  # Find all links with an href attribute
                    if links:
                        last_link = links[-1]  # Get the last link
                        last_link_url = last_link['href']
                        print(f"✅ Last Found Link: {last_link_url}")

                        # Open the link in the browser
                        driver.get(last_link_url)
                        time.sleep(5)  # Wait for the page to load

                    # If the search_name is "Faculty", check if the section <div class="block has-aside"> exists
                        if search_name == "Faculty":
                            try:
                                section = driver.find_element(By.CSS_SELECTOR, "div.block.has-aside")
                                print("✅ Found section <div class='block has-aside'> in the linked page.")

                                # Extract content using BeautifulSoup
                                page_html = driver.page_source
                                page_soup = BeautifulSoup(page_html, 'html.parser')

                                # Extract everything inside the <div class="block has-aside">
                                section_content = page_soup.find('div', class_='block has-aside')
                                #print("\n--- Extracted Section Content ---")
                                #print(section_content.get_text(separator=' ', strip=True))
                                extracted_text = section_content.get_text(separator=' ', strip=True)

                            except Exception as e:
                                print("Section <div class='block has-aside'> not found. Extracting content from the first page.")
                                # If the section is not found, extract the content from the first page
                                #print(f"--- Extracted Content from the first page ---\n{content_text}")
                                extracted_text = content_text
                        else:
                            # If search_name is "Study plan", extract the entire content of the linked page
                            # Extract the <main> element
                            main_element = driver.find_element(By.TAG_NAME, "main")  # Locate the <main> section
                            main_html = main_element.get_attribute('outerHTML')  # Get the HTML inside <main>
                            page_soup = BeautifulSoup(main_html, 'html.parser')  # Parse with BeautifulSoup

                            # REMOVE THE BROCHURE AND COUNTRIES SECTIONS
                            unwanted_section = page_soup.find("div", class_="hero-article__info")
                            if unwanted_section:
                                unwanted_section.decompose()  

                            # Extract the content from the linked page without checking for the section
                            #print(f"--- Extracted Content from the linked page for {search_name} ---")
                            #print(page_soup.get_text(separator=' ', strip=True))
                            # Extract the content from the linked page
                            extracted_text = page_soup.get_text(separator=' ', strip=True)

                    else:
                        print("No links found in this article.")
                        extracted_text = content_text

                    # **Save extracted text to a .txt file**
                    file_name = f"{search_name.replace(' ', '_')}.txt"
                    file_path = os.path.join(postgraduates_masters_dir, file_name)

                    with open(file_path, "w", encoding="utf-8") as file:
                        file.write(extracted_text)

                    print(f"✅ Extracted content saved to: {file_path}")

                    break  # Stop after finding the first match for this search_name

            except Exception as e:
                print(f"Skipping article due to error: {e}")

    else:
        print("No articles found inside 'block has-aside'.")

except Exception as e:
    print(f"Error locating elements: {e}")

# Close WebDriver
driver.quit()

Error locating elements: Message: 
Stacktrace:
	GetHandleVerifier [0x0095C7F3+24435]
	(No symbol) [0x008E2074]
	(No symbol) [0x007B06E3]
	(No symbol) [0x007F8B39]
	(No symbol) [0x007F8E8B]
	(No symbol) [0x00841AC2]
	(No symbol) [0x0081D804]
	(No symbol) [0x0083F20A]
	(No symbol) [0x0081D5B6]
	(No symbol) [0x007EC54F]
	(No symbol) [0x007ED894]
	GetHandleVerifier [0x00C670A3+3213347]
	GetHandleVerifier [0x00C7B0C9+3295305]
	GetHandleVerifier [0x00C7558C+3271948]
	GetHandleVerifier [0x009F7360+658144]
	(No symbol) [0x008EB27D]
	(No symbol) [0x008E8208]
	(No symbol) [0x008E83A9]
	(No symbol) [0x008DAAC0]
	BaseThreadInitThunk [0x769FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778382AE+286]
	RtlGetAppContainerNamedObjectPath [0x7783827E+238]



#### Function to extract faculty and study plan information

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_faculty_and_studyplan(url, search_name, save_directory):
    """
    Extracts text content from the specified search section on the webpage
    and saves it as a .txt file in the specified directory.

    :param url: The webpage URL to scrape.
    :param search_name: The section name to search for (e.g., "Faculty", "Study plan").
    :param save_directory: The directory where the extracted text will be saved.
    """
    
    # Ensure the save directory exists
    os.makedirs(save_directory, exist_ok=True)

    # Setup WebDriver
    driver_path = ChromeDriverManager().install()
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)
    driver.get(url)

    # Wait for elements to load
    wait = WebDriverWait(driver, 10)

    try:
        # Find the main div dynamically
        main_block = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.block.has-aside")))

        # Find all articles within the main block
        articles = main_block.find_elements(By.CSS_SELECTOR, "article.content__article")

        if articles:
            for article in articles:
                try:
                    # Find the <h2> heading inside the article
                    heading_element = article.find_element(By.CSS_SELECTOR, "h2.content__heading.has-border.beta")
                    heading_text = heading_element.text.strip()
                    
                    if search_name in heading_text:
                        print(f"✅ Found Article: {heading_text}")

                        # Extract the corresponding content within <div class="content__article-wrap">
                        content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                        content_html = content_element.get_attribute('outerHTML')

                        # Parse the content using BeautifulSoup
                        soup = BeautifulSoup(content_html, 'html.parser')
                        content_text = soup.get_text(separator=' ', strip=True)

                        # Check for links inside the content and extract the last one
                        links = soup.find_all('a', href=True)
                        if links:
                            last_link = links[-1]['href']  # Get the last link URL
                            print(f"✅ Last Found Link: {last_link}")

                            # Open the linked page
                            driver.get(last_link)
                            time.sleep(5)  # Wait for the page to load

                            if search_name == "Faculty":
                                try:
                                    section = driver.find_element(By.CSS_SELECTOR, "div.block.has-aside")
                                    print("✅ Found section <div class='block has-aside'> in the linked page.")

                                    # Extract content using BeautifulSoup
                                    page_soup = BeautifulSoup(driver.page_source, 'html.parser')
                                    section_content = page_soup.find('div', class_='block has-aside')
                                    extracted_text = section_content.get_text(separator=' ', strip=True)

                                except Exception:
                                    print("Section <div class='block has-aside'> not found. Extracting content from the first page.")
                                    extracted_text = content_text

                            else:
                                # If search_name is "Study plan", extract the entire content of the linked page
                                main_element = driver.find_element(By.TAG_NAME, "main")
                                main_html = main_element.get_attribute('outerHTML')
                                page_soup = BeautifulSoup(main_html, 'html.parser')

                                # Remove brochure/country sections if present
                                unwanted_section = page_soup.find("div", class_="hero-article__info")
                                if unwanted_section:
                                    unwanted_section.decompose()  

                                extracted_text = page_soup.get_text(separator=' ', strip=True)

                        else:
                            print("No links found in this article.")
                            extracted_text = content_text

                        # Save extracted text to a .txt file
                        file_name = f"{search_name.replace(' ', '_')}.txt"
                        file_path = os.path.join(save_directory, file_name)

                        with open(file_path, "w", encoding="utf-8") as file:
                            file.write(extracted_text)

                        print(f"✅ Extracted content saved to: {file_path}")
                        break  # Stop after finding the first match for this search_name

                except Exception as e:
                    print(f"Skipping article due to error: {e}")

        else:
            print("No articles found inside 'block has-aside'.")

    except Exception as e:
        print(f"Error locating elements: {e}")

    finally:
        # Close WebDriver
        driver.quit()

In [5]:
de.extract_faculty_and_studyplan(
    url="https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/",
    search_name="Faculty",
    save_directory=r"..\..\data\Webscrapping\test"
)

de.extract_faculty_and_studyplan(
    url="https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/",
    search_name="Study plan",
    save_directory=r"..\..\data\Webscrapping\test"
)


✅ Found Article: Faculty
✅ Last Found Link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/faculty/
Section <div class='block has-aside'> not found. Extracting content from the first page.
✅ Extracted content saved to: ..\..\data\Webscrapping\test\Faculty.txt
✅ Found Article: Study plan
✅ Last Found Link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-risk-analysis-and-management/study-plan/
✅ Extracted content saved to: ..\..\data\Webscrapping\test\Study_plan.txt


**Conclusions:**
- The function created works very well to select the specific content__article, identify if there are links, if yes save the content of the links in a file, if not extract the study plan or faculty section from the main page and save in a file

### FAQs - Very general through all postgraduate and master programs

In [9]:
import os
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_faq_section(url, save_directory, search_name="FAQ"):
    """
    Extracts the FAQ section from a given webpage and saves it as 'FAQ_postgraduate_master_degrees.txt'.

    :param url: The webpage URL to scrape.
    :param save_directory: The directory where the extracted text will be saved.
    :param search_name: The section name to search for (default is "FAQ").
    """

    # Setup WebDriver
    driver_path = ChromeDriverManager().install()
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)

    try:
        # Open the target webpage
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Find the main block containing articles
        main_block = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "block.has-aside")))
        articles = main_block.find_elements(By.CLASS_NAME, "content__article")

        extracted_text = None

        if articles:
            for article in articles:
                try:
                    # Find the article's <h2> heading
                    heading_element = article.find_element(By.CLASS_NAME, "content__heading")
                    heading_text = heading_element.text.strip()

                    # Extract text only for the specified search section
                    if search_name in heading_text:
                        print(f"✅ Found Article: {heading_text}")

                        # Extract the content within <div class="content__article-wrap">
                        content_element = article.find_element(By.CSS_SELECTOR, "div.content__article-wrap")
                        content_html = content_element.get_attribute('outerHTML')

                        # Parse the content using BeautifulSoup
                        soup = BeautifulSoup(content_html, 'html.parser')

                        # Extract the text from the content
                        extracted_text = soup.get_text(separator=' ', strip=True)
                        print(f"--- Extracted Text ---\n{extracted_text}")

                        break  # Stop after finding the first match
                except Exception as e:
                    print(f"Skipping article due to error: {e}")

        else:
            print("No articles found inside 'block has-aside'.")

        if extracted_text:
            # Ensure save directory exists
            os.makedirs(save_directory, exist_ok=True)

            # Define file name
            file_name = "FAQ_postgraduate_master_degrees.txt"
            file_path = os.path.join(save_directory, file_name)

            # Save extracted text to a file
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(extracted_text)

            print(f"✅ Extracted content saved to: {file_path}")

    except Exception as e:
        print(f"Error occurred: {e}")

    finally:
        # Close WebDriver
        driver.quit()


# Example usage:
extract_faq_section(
    url="https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-business-analytics/",
    save_directory=r"..\..\data\Webscrapping\postgraduate_master_degrees"
)

✅ Found Article: FAQ
--- Extracted Text ---
FAQ Do you have a question? Explore the most frequently asked questions and find the answer here! If you still have unanswered questions, please contact us! Applications When can I apply? It is only possible to submit the application during its applications phase. The date of the call for applications is available in the Admissions and Fees page of each program. However, you can fill the application form at any time, submitting it when the applications phase starts. How can I apply? The applications are submitted online, in NOVA IMS’ Applications Portal . To submit their application, the applicants must register in the Portal, fill the form, upload your Curriculum Vitae, choose the program and pay the application fee in the Portal. The application process is the same for all the applicants, national or international. Which documents are required? The only mandatory document during the application process is the Curriculum Vitae (in .doc or .p

**Conclusions:**
- The FAQ section is the same for every postgraduate and master course, so its seperated and saved in only one file

### MAIN PAGE - Postgraduate and Masters

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-business-analytics/"

# Open the target webpage
driver.get(url)
wait = WebDriverWait(driver, 10)


# Find the main block containing articles
main_block = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "block.has-aside")))

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# REMOVE THE BROCHURE AND COUNTRIES SECTIONS
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  
    

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)

In [10]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-business-analytics/"
driver.get(url)
wait = WebDriverWait(driver, 10)

try:
    # Find the <main> section
    main_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/main")))

    # Get all <div> elements inside <main>
    div_blocks = main_element.find_elements(By.XPATH, "./div")

    extracted_html = ""

    for div in div_blocks:
        class_name = div.get_attribute("class")

        # Stop before the block.has-aside section
        if "block has-aside" in class_name:
            break

        # Append HTML content
        extracted_html += div.get_attribute("outerHTML") + "\n"

finally:
    # Close WebDriver
    driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(extracted_html, "html.parser")

# REMOVE the Brochure and Countries sections
for unwanted_class in ["hero-article__info", "countries-section"]:
    unwanted_section = soup.find("div", class_=unwanted_class)
    if unwanted_section:
        unwanted_section.decompose()

# Extract and clean text
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)


Master's Degree Program in Data Science and Advanced Analytics,
with a specialization in Business Analytics
en
Education
Programs
Postgraduate Programs and Master Degree Programs
Master Degree Program in Data Science and Advanced Analytics, with a specialization in Business Analytics
The Master degree Program in Data Science and Advanced Analytics, with a specialization in Business Analytics, is aimed at
market oriented people
,
who want to apply effective analytical models to different business problems, interpreting the results and their implications to the business, with the objective of taking data-driven decisions to optimize the business process
.
In every academic year, the partners offer a paid internship to the 1st year students, to be undertaken during the 2nd year. 1st year students should apply to the internship(s) they are interested in. The internship will be assigned to the student gathering the best qualification in the application.
This Master is ranked, for the 2
nd
t

In [11]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-business-analytics/"
driver.get(url)
wait = WebDriverWait(driver, 10)

try:
    # Find the <main> section
    main_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/main")))

    # Get all <div> elements inside <main>
    div_blocks = main_element.find_elements(By.XPATH, "./div")

    extracted_html = ""

    # Extract content **before** block.has-aside
    for div in div_blocks:
        class_name = div.get_attribute("class")

        if "block has-aside" in class_name:
            break  # Stop before the block.has-aside section

        extracted_html += div.get_attribute("outerHTML") + "\n"

    # Extract block.has-aside content
    has_aside_block = driver.find_element(By.CLASS_NAME, "block.has-aside")
    articles = has_aside_block.find_elements(By.CLASS_NAME, "content__article")

    relevant_headings = ["Program Coordinator", "Length, timetable and exams", "Admissions and fees"]
    extracted_aside_html = ""

    for article in articles:
        try:
            # Extract the heading
            heading_element = article.find_element(By.CLASS_NAME, "content__heading")
            heading_text = heading_element.text.strip()

            if heading_text in relevant_headings:
                extracted_aside_html += article.get_attribute("outerHTML") + "\n"
        except:
            pass  # Skip if element not found

finally:
    # Close WebDriver
    driver.quit()

# Parse with BeautifulSoup
soup_main = BeautifulSoup(extracted_html, "html.parser")
soup_aside = BeautifulSoup(extracted_aside_html, "html.parser")

# REMOVE Brochure and Countries sections
for unwanted_class in ["hero-article__info", "countries-section"]:
    unwanted_section = soup_main.find("div", class_=unwanted_class)
    if unwanted_section:
        unwanted_section.decompose()

# Extract and clean text
main_text = soup_main.get_text(separator="\n", strip=True)
aside_text = soup_aside.get_text(separator="\n", strip=True)

# Print extracted text
print("\n📌 **Main Content (Before block.has-aside)**:\n")
print(main_text)

print("\n📌 **Extracted block.has-aside Content (Selected Sections)**:\n")
print(aside_text)



📌 **Main Content (Before block.has-aside)**:

Master's Degree Program in Data Science and Advanced Analytics,
with a specialization in Business Analytics
en
Education
Programs
Postgraduate Programs and Master Degree Programs
Master Degree Program in Data Science and Advanced Analytics, with a specialization in Business Analytics
The Master degree Program in Data Science and Advanced Analytics, with a specialization in Business Analytics, is aimed at
market oriented people
,
who want to apply effective analytical models to different business problems, interpreting the results and their implications to the business, with the objective of taking data-driven decisions to optimize the business process
.
In every academic year, the partners offer a paid internship to the 1st year students, to be undertaken during the 2nd year. 1st year students should apply to the internship(s) they are interested in. The internship will be assigned to the student gathering the best qualification in the app

#### Function for MAIN PAGE extraction

In [ ]:
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

def extract_main_course(url, save_directory):
    # Setup WebDriver
    driver_path = ChromeDriverManager().install()
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service)

    try:
        # Open the target URL
        driver.get(url)
        wait = WebDriverWait(driver, 10)

        # Extract the last part of the URL for the file name
        file_name = url.split("/")[-2] + "_main_course.txt"
        file_path = os.path.join(save_directory, file_name)

        # Find the <main> section
        main_element = wait.until(EC.presence_of_element_located((By.XPATH, "/html/body/main")))

        # Get all <div> elements inside <main>
        div_blocks = main_element.find_elements(By.XPATH, "./div")

        extracted_html = ""

        # Extract content **before** block.has-aside
        for div in div_blocks:
            class_name = div.get_attribute("class")

            if "block has-aside" in class_name:
                break  # Stop before the block.has-aside section

            extracted_html += div.get_attribute("outerHTML") + "\n"

        # Extract block.has-aside content
        has_aside_block = driver.find_element(By.CLASS_NAME, "block.has-aside")
        articles = has_aside_block.find_elements(By.CLASS_NAME, "content__article")

        relevant_headings = ["Program Coordinator", "Length, timetable and exams", "Admissions and fees"]
        extracted_aside_html = ""

        for article in articles:
            try:
                # Extract the heading
                heading_element = article.find_element(By.CLASS_NAME, "content__heading")
                heading_text = heading_element.text.strip()

                if heading_text in relevant_headings:
                    extracted_aside_html += article.get_attribute("outerHTML") + "\n"
            except:
                pass  # Skip if element not found

        # Close WebDriver after extraction
        driver.quit()

        # Parse with BeautifulSoup
        soup_main = BeautifulSoup(extracted_html, "html.parser")
        soup_aside = BeautifulSoup(extracted_aside_html, "html.parser")

        # REMOVE Brochure and Countries sections
        for unwanted_class in ["hero-article__info", "countries-section"]:
            unwanted_section = soup_main.find("div", class_=unwanted_class)
            if unwanted_section:
                unwanted_section.decompose()

        # Extract and clean text
        main_text = soup_main.get_text(separator="\n", strip=True)
        aside_text = soup_aside.get_text(separator="\n", strip=True)

        # Print out relevant progress messages
        print("✅ Main Content Extracted Successfully")

        print("✅ Extracted block.has-aside Content (Selected Sections)")

        # Save the extracted content to a .txt file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(main_text)
            file.write("\n\n---\n\n")
            file.write(aside_text)

        print(f"✅ Content saved to {file_path}")

    except Exception as e:
        print(f"❌ **Error occurred: {e}")
    finally:
        # Ensure the WebDriver is closed in case of an error
        driver.quit()




In [ ]:
# Example Usage:
# URL of the course page and path where the file should be saved
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-business-analytics/"
save_directory = r"..\..\data\Webscrapping\test"

# Call the function
de.extract_main_course(url, save_directory)

✅ Main Content Extracted Successfully
✅ Extracted block.has-aside Content (Selected Sections)
✅ Content saved to ..\..\data\Webscrapping\test\master-degree-program-in-data-science-and-advanced-analytics-with-a-specialization-in-business-analytics_main_course.txt


**Conclusions:**
- For main page I'm extracting everything besides Study Plan, Faculty, FAQ, Academic Calender, Scholarships, Achievement Awards

## Pipeline for Postgraduate and Master Degrees

### OLD CODE

I'm ignoring at the moment schedules and timetables

In [5]:
# Check if the dictionary is not empty
if dict_programs_links:
    # Get the first key (URL) and its value (filtered links)
    first_url, first_filtered_links = next(iter(dict_programs_links.items()))
    
    # Print the first URL and filtered links
    print(f"First URL: {first_url}")
    print(f"Filtered links for the first URL: {first_filtered_links}")

    # Extract the first three elements from the dictionary
    first_three_programs = list(dict_programs_links.items())[:3]

    # Iterate over the first three programs
    for program_url, program_links in first_three_programs:
        # Extract the name of the bachelor degree from the URL
        posgrad_master_degree_name = program_url.split('/')[-2]

        # Substitute the name in the links
        links_to_extract = [
            f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/program/#",
            f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/teaching-staff/#"
            
        ]

        # Print the substituted links
        print(f"Substituted links for {posgrad_master_degree_name}: {links_to_extract}")

        # Optionally, interact with the page after it loads
else:
    print("The dictionary is empty.")


#### Main code 

In [16]:
import re
# Directory for storing extracted text
postgraduates_masters_dir = r"..\..\data\Webscrapping\postgraduate_master_degrees"
os.makedirs(postgraduates_masters_dir, exist_ok=True)  # Ensure directory exists

In [17]:
def sanitize_filename(filename):
    """Sanitize the filename to remove any invalid characters."""
    # Replace invalid characters with underscores
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

def check_page_exists(url):
    """Returns True if the page contains a <main> element, otherwise False."""
    try:
        driver.get(url)
        time.sleep(3)  # Allow page to load
        driver.find_element(By.TAG_NAME, "main")  # Try finding <main>
        return True  # If found, page exists
    except NoSuchElementException:
        return False  # <main> not found, assume page doesn't exist
    except TimeoutException:
        return False  # Page timeout, assume it doesn't exist
    except Exception as e:
        print(f"Error checking {url}: {e}")
        return False  # Catch-all for unexpected errors

def extract_main_text(url, remove_teaching_staff, remove_study_plan):
    """Extracts the main content text from a given URL, adjusting section removals dynamically."""
    driver.get(url)
    time.sleep(5)  # Wait for page load

    try:
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_html = main_element.get_attribute('outerHTML')
        soup = BeautifulSoup(main_html, "html.parser")

        # Adjust which sections to remove
        if not remove_study_plan and not remove_teaching_staff:
            remove_indices = [0, 1, 2]  # Default: remove Study Plan (1), Teaching Staff (2), Academic Calendar (3)
        elif not remove_study_plan:
            remove_indices = [0, 2]  # Remove First Section (0) and Teaching Staff (2)
        else:
            remove_indices = [2]  # Remove only Teaching Staff (2)

        # Remove unwanted sections
        for selector in [
            ("div", "hero-article__info", None),  # Remove Brochure and Country Sections
            ("article", "content__article", remove_indices)
        ]:
            elements = soup.find_all(selector[0], class_=selector[1])
            if selector[2]:  # If we have specific indices to remove
                for index in selector[2]:
                    if index < len(elements):
                        elements[index].decompose()
            elif elements:  # Remove the single found element
                elements[0].decompose()

        return soup.get_text(separator="\n", strip=True)
    except NoSuchElementException:
        print(f"Error extracting text from {url}: <main> element not found.")
        return ""
    except Exception as e:
        print(f"Error extracting text from {url}: {e}")
        return ""

In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)


# Process second URL in `dict_programs_links`
if len(dict_programs_links) > 1:
    second_url, second_filtered_links = list(dict_programs_links.items())[1]
    print(f"Processing: {second_url}")

    for program_url in second_filtered_links:
        posgrad_master_degree_name = program_url.split('/')[-2]  # Extract name from URL

        # URLs to check
        teaching_staff_url = f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/teaching-staff/#"
        study_plan_url = f"https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/{posgrad_master_degree_name}/study-plan/#"

        # Check if these pages exist
        study_plan_exists = check_page_exists(study_plan_url)
        teaching_staff_exists = check_page_exists(teaching_staff_url)

        # Extract and save main course text
        main_text = extract_main_text(program_url, not teaching_staff_exists, not study_plan_exists)
        if main_text:
            # Sanitize posgrad_master_degree_name to ensure it's a valid file name
            sanitized_name = sanitize_filename(posgrad_master_degree_name)
            
            # Construct file path with sanitized name
            file_path = os.path.join(postgraduates_masters_dir, f"{sanitized_name}_main_course_extracted_text.txt")
            
            # Write extracted text to the file
            with open(file_path, "w", encoding="utf-8") as file:
                file.write(f"Text from {program_url}:\n{main_text}\n\n")

        # Extract and save text from study-plan & teaching-staff pages
        for url, exists in [(study_plan_url, study_plan_exists), (teaching_staff_url, teaching_staff_exists)]:
            if exists:
                extracted_text = extract_main_text(url, not teaching_staff_exists, not study_plan_exists)
                if extracted_text:
                    # Sanitize posgrad_master_degree_name and page-specific name
                    sanitized_name = sanitize_filename(posgrad_master_degree_name)
                    page_name = url.split('/')[-2]  # Extract page name (study-plan, teaching-staff)
                    file_name = os.path.join(postgraduates_masters_dir, f"{sanitized_name}_{page_name}_extracted_text.txt")
                    
                    # Write extracted text to the file
                    with open(file_name, "w", encoding="utf-8") as file:
                        file.write(f"Text from {url}:\n{extracted_text}\n\n")

    print("Text extraction completed and saved.")
else:
    print("The dictionary is empty.")

# Close WebDriver
driver.quit()


In [ ]:
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-science-for-finance/"
# Define the save directory (you can modify this as needed)
save_directory=r"..\..\data\Webscrapping\test"
de.extract_main_course(url, save_directory=save_directory)

In [6]:
url = "https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-data-science-for-finance/"
# Define the save directory (you can modify this as needed)
save_directory=r"..\..\data\Webscrapping\test"
de.extract_faculty_and_studyplan(url, search_name="Study plan", save_directory=save_directory)

✅ Found Article: Study plan
✅ Last Found Link: https://www.best-masters.com/ranking-master-insurance-risk-actuarial-sciences.html
Last link is not valid for Faculty or Study Plan. Skipping link.
✅ Extracted content saved to: ..\..\data\Webscrapping\test\postgraduate-program-in-data-science-for-finance_Study plan.txt


### Extract postgraduate and masters data

In [13]:
# Iterate over the filtered links and apply your functions
for link in second_filtered_links:
   
    # Define the save directory (you can modify this as needed)
    save_directory=r"..\..\data\Webscrapping\postgraduate_master_degrees"

    # Extract the faculty and study plan
    de.extract_faculty_and_studyplan(link, search_name="Faculty", save_directory=save_directory)
    #de.extract_faculty_and_studyplan(link, search_name="Study plan", save_directory=save_directory)
    
    # Extract the FAQ section
    #de.extract_faq_section(link[0], save_directory=save_directory, search_name="FAQ")
    
    # Extract the main course section
    #de.extract_main_course(link, save_directory=save_directory)

Attempt 1: Accessing https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/
✅ Successfully accessed https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/ for Faculty
⚠️ Not Found Article: Study plan
⚠️ Not Found Article: NOVA IMS' Program Coordinator
✅ Found Article: Faculty
✅ Last Found Link: https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-intelligence/teaching-staff/
✅ Faculty link page is not empty. Extracting contect from link page.
✅ Extracted content saved to: ..\..\data\Webscrapping\postgraduate_master_degrees\postgraduate-program-in-business-intelligence_Faculty.txt
Attempt 1: Accessing https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/postgraduate-program-in-business-int